In [1]:
import tensorflow as tf

In [16]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, concatenate,BatchNormalization,Conv2D, Add, Activation,MaxPool2D
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Model


In [3]:
input_a = Input(shape= [1], name = 'Wide_Input')
input_b = Input(shape= [1], name = 'Deep_Input')

In [5]:
layer_1 = Dense(units= 30, activation = 'relu')(input_b)
layer_2 = Dense(units = 30, activation = 'relu')(layer_1)

concat = concatenate([input_a, layer_2])
output = Dense(1, name = 'Outputt')(concat)


aux_out = Dense(1, name = 'Aux_out')(layer_2)

model = Model(inputs = [input_a, input_b], outputs = [output, aux_out])

In [6]:
plot_model(model)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [7]:
# inherit from the Model base class
class WideAndDeepModel(Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        '''initializes the instance attributes'''
        super().__init__(**kwargs)
        self.hidden1 = Dense(units, activation=activation)
        self.hidden2 = Dense(units, activation=activation)
        self.main_output = Dense(1)
        self.aux_output = Dense(1)

    def call(self, inputs):
        '''defines the network architecture'''
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        
        return main_output, aux_output


In [8]:
# create an instance of the model
model = WideAndDeepModel()

# Residual Network

In [11]:
import tensorflow_datasets as tfds

In [20]:
class IdentityBlock(Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name= '')
        
        self.conv1 = Conv2D(filters, kernel_size, padding= 'same')
        self.bn1 = BatchNormalization()
        
        self.conv2 =  Conv2D(filters, kernel_size, padding = 'same')
        self.bn2 = BatchNormalization()
        
        self.act = Activation('relu')# YOUR CODE HERE
        self.add = Add()
        
        
    def call(self, inputs_tensor):
        x = self.conv1(inputs_tensor)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act(x)
        x = self.add([x, inputs_tensor])
        x = self.act(x)
        
        return x
        

In [21]:
class ResNet(Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        
        self.conv = Conv2D(64, 7, padding = 'same')
        self.bn = BatchNormalization()
        self.act = Activation('relu')
        
        self.max_pool = MaxPool2D((3,3))
        
        self.id1a = IdentityBlock(64,3)
        self.id2a = IdentityBlock(64,3)
        
        self.global_pool= GlobalAveragePooling2D()
        
        self.classifier = Dense(num_classes, activation = 'softmax')
        
        
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)
        
        x = self.id1a(x)
        x = self.id2a(x)
        
        x = self.global_pool(x)
        return self.classifier(x)

In [22]:
# utility function to normalize the images and return (image, label) pairs.
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

In [23]:
resnet = ResNet(10)


In [24]:
dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to ./data/mnist/3.0.1. Subsequent calls will reuse this data.


In [28]:
dataset = dataset.map(preprocess).batch(32)

In [32]:
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet.fit(dataset, epochs=1)

1875/1875 [==============================] - 46s 24ms/step - loss: 0.1801 - accuracy: 0.9568


# Exercise VGG Network

In [37]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
        for i in range(self.repetitions):
            
            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters, self.kernel_size, activation = 'relu',padding= 'same')
        
        # Define the max pool layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=(pool_size), strides=strides)
  
    def call(self, inputs):
        # access the class's conv2D_0 layer
        conv2D_0 = self.conv2D_0
        
        # Connect the conv2D_0 layer to inputs
        x = conv2D_0(inputs)

        # for the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
        for i in range(1,self.repetitions):
            # access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
            conv2D_i = vars(self)[f'conv2D_{i}']
            
            # Use the conv2D_i and connect it to the previous layer
            x = conv2D_i(x)

        # Finally, add the max_pool layer
        max_pool = self.max_pool(x)
        
        return max_pool

In [47]:
class MyVGG(Model):
    def __init__(self, num_classes):
        super(MyVGG, self).__init__()
        
        self.block_a = Block(64, 3,2)
        self.block_b = Block(128, 3, 2)
        self.block_c = Block(256, 3, 3)
        self.block_d = Block(512, 3, 3)
        self.block_e = Block(512, 3, 3)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(256, activation = 'relu')
        self.classifier = tf.keras.layers.Dense(num_classes, activation = 'softmax')
        
    def call(self, inputs):
        
        x = self.block_a(inputs)
        x = self.block_b(x)   
        x = self.block_c(x) 
        x = self.block_d(x) 
        x = self.block_e(x) 
        x = self.flatten(x) 
        x = self.fc(x) 
        x = self.classifier(x)
        return x

In [48]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes 

In [49]:
vgg = MyVGG(num_classes=2)

In [50]:
# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)
# Compile with losses and metrics

In [51]:
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [52]:
# Train the custom VGG model
vgg.fit(dataset, epochs=10)

Epoch 1/10
727/727 [==============================] - 731s 1s/step - loss: 0.6885 - accuracy: 0.5493
Epoch 2/10
727/727 [==============================] - 725s 997ms/step - loss: 0.6649 - accuracy: 0.6182
Epoch 3/10
727/727 [==============================] - 724s 996ms/step - loss: 0.6392 - accuracy: 0.6524
Epoch 4/10
727/727 [==============================] - 723s 994ms/step - loss: 0.6180 - accuracy: 0.6717
Epoch 5/10
727/727 [==============================] - 725s 997ms/step - loss: 0.6024 - accuracy: 0.6820
Epoch 6/10
727/727 [==============================] - 724s 996ms/step - loss: 0.5907 - accuracy: 0.6910
Epoch 7/10
727/727 [==============================] - 722s 993ms/step - loss: 0.5814 - accuracy: 0.6992
Epoch 8/10
727/727 [==============================] - 728s 1s/step - loss: 0.5733 - accuracy: 0.7052
Epoch 9/10
727/727 [==============================] - 738s 1s/step - loss: 0.5661 - accuracy: 0.7098
Epoch 10/10
727/727 [==============================] - 740s 1s/step - los